# Monty Hall simulations

In this notebook we'll simulate *lots* of trials of the game described in the Monty Hall problem and see how it turns out.

In [47]:
# Import various packages to gain useful pythonic powers
import random
import numpy as np
import pandas as pd

### Basic setup

In the spirit of the Python programming language, we'll imagine that the doors are numbered 0, 1, 2. Then, to simulate a run through the game, we'll:
 * Choose a door at random to be the one with the prize behind it.
 * Choose a door at random to be the players's initial choice.
 * Choose a door for Monty to open a door that (a) isn't the player's choice and (b) doesn't hide the prize.
 * Apply a strategy, either *stick* or *switch*
 * See whether the player wins.

I'll store all these things in long lists so that
* `prizeDoor[j]` will be the position of the prize in the `j`-th trial;
* `initialChoice[j]` will be the player's first choice in the `j`-th trial;
* `montyOpens[j]` will be the door Monty opens in the `j`-th trial;
* `finalChoice[j]` will be the door that the player is left with after applying her strategy
* `result[j]` will be 1 if the player won in the `j`-th trial and 0 otherwise.

In [51]:
# Set up all the lists
nTrials = 1000
prizeDoor = np.zeros( nTrials, dtype=int )
initialChoice = np.zeros( nTrials, dtype=int )
montyOpens = np.zeros( nTrials, dtype=int )
finalChoice = np.zeros( nTrials, dtype=int )
result = np.zeros( nTrials, dtype=int )

In [52]:
# Specify the door numbers. Here we use Python's
# built-in Set objects so that it is easy to,
# for example, find those doors that
# (a) the punter hasn't chosen
# (b) don't hide the prize.
doorNumSet = { 0, 1, 2 } # The {} braces enclose the elements of the set
doorNumTuple = tuple( doorNumSet ) # Needed for random sampling

### First strategy: Stick

In [54]:
# Run the trials
for j in range(nTrials):
    # Make the two random choices
    # We call random.choice() on a tuple because it doesn't work on sets.
    prizeDoor[j] = random.choice( doorNumTuple )
    initialChoice[j] = np.random.choice( doorNumTuple )

    # Work out which doors don't hide the prize and open one of them.
    # Also, note which door remains unopened.
    nonPrizeDoors = doorNumSet - {prizeDoor[j]}
    nonChosenDoors = doorNumSet - {initialChoice[j]}
    doorsMontyCouldOpen = nonPrizeDoors.intersection(nonChosenDoors)
    montyOpens[j] = random.choice( tuple(doorsMontyCouldOpen) )
                                
    # Apply the strategy, in this case "stick"
    finalChoice[j] = initialChoice[j]

    # Determine the result
    if finalChoice[j] == prizeDoor[j]:
        result[j] = 1
    else:
        result[j] = 0 
    
# Assemble the results into a nice table and display a few rows
stick_df = pd.DataFrame( {
    'Prize':prizeDoor, 
    'InitialChoice':initialChoice, 
    'MontyOpens':montyOpens, 
    'FinalChoice':finalChoice, 
    'Result':result
} )

stick_df.head(10)

,Prize,InitialChoice,MontyOpens,FinalChoice,Result
0,0,2,1,2,0
1,2,2,0,2,1
2,1,2,0,2,0
3,1,2,0,2,0
4,0,1,2,1,0
5,2,1,0,1,0
6,1,1,0,1,1
7,2,1,0,1,0
8,2,2,0,2,1
9,2,2,1,2,1


In [55]:
# Report the number of times we won
nStickWins = np.sum( result )
print( "With the strategy \"stick\", got ", nStickWins, "wins in", nTrials, "trials." ) ;

With the strategy "stick", got  336 wins in 1000 trials.


### Second strategy: Switch

In [57]:
# Run the trials
for j in range(nTrials):
    # Make the two random choices
    prizeDoor[j] = random.choice( doorNumTuple )
    initialChoice[j] = np.random.choice( doorNumTuple )

    # Work out which doors don't hide the prize and open one of them.
    # Also, note which door remains unopened.
    nonPrizeDoors = doorNumSet - {prizeDoor[j]}
    nonChosenDoors = doorNumSet - {initialChoice[j]}
    montyOpens[j] = random.choice( tuple(nonPrizeDoors.intersection(nonChosenDoors)) )
                                
    # Apply the strategy, in this case "switch"
    remainingDoorSet = doorNumSet - { initialChoice[j], montyOpens[j] }
    finalChoice[j] = remainingDoorSet.pop() # pop() returns the sole element in remainingDoorSet

    # Determine the result
    if finalChoice[j] == prizeDoor[j]:
        result[j] = 1
    else:
        result[j] = 0 
    
# Assemble the results into a nice table and dispay a few rows
switch_df = pd.DataFrame( {
    'Prize':prizeDoor, 
    'InitialChoice':initialChoice, 
    'MontyOpens':montyOpens, 
    'FinalChoice':finalChoice, 
    'Result':result
} )

switch_df.head(10)

,Prize,InitialChoice,MontyOpens,FinalChoice,Result
0,2,0,1,2,1
1,1,1,2,0,0
2,0,0,2,1,0
3,0,2,1,0,1
4,2,2,0,1,0
5,1,1,0,2,0
6,1,1,0,2,0
7,0,0,1,2,0
8,1,0,2,1,1
9,0,0,2,1,0


In [58]:
# Report the number of times we won
nSwitchWins = np.sum( result )
print( "With the strategy \"switch\", got ", nSwitchWins, "wins in", nTrials, "trials." ) ;

With the strategy "switch", got  652 wins in 1000 trials.
